# 1. 分析过程
#### 识别客户价值应用最广泛的模型是通过3个指标（最近消费时间间隔（Recency）、消费频率（Frequency）和消费金额（Money））来进行客户细分，识别出高价值客户，简称RFM模型。

RFM模型中，消费金额表示一段时间内，客户购买企业产品金额的总和。由于航空票价受到运输距离、舱位等级等多种因素影响，同样消费金额的不同旅客对航空公司的价值是不同的。例如，一位购买长航线、低等舱位票的旅客与一位购买短航线、高等级舱位票的旅客相比，后者对于航空公司的价值可能更高。因此，这个指标并不适合航空公司客户价值分析。我们选择客户在一定时间内累积的飞行里程M和客户在一定时间内乘坐舱位所对应的折扣系数的平均值C两个指标代替消费金额。此外，还考虑航空公司会员入会时间的长短在一定程度上影响客户价值，所以在模型中增加客户关系长度L，作为区分客户的另一指标。


综上所述，航空公司识别客户价值模型（LRFMC）指标如下：


#### 客户关系长度L：会员入会时间距观测窗口结束的月份

#### 消费时间间隔R：客户最近一次乘坐公司飞机距观测窗口结束的月数

#### 消费频率F：客户在观测窗口内乘坐公司飞机的次数

#### 飞行里程M：客户在观测窗口内飞行里程

#### 折扣系数的平均值C：客户在观测窗口内乘坐舱位所对应的折扣系数的平均值


 传统的RFM模型采用属性分箱的方法，如下图所示（依据属性的平均值进行划分），虽然也能识别出最有价值客户，但是如果LRFMC模型使用同样的方法，则细分的属性太多，提高了后续根据客户类别的营销成本。故本次分析使用聚类方法，通过对航空公司客户价值的五个指标进行Kmeans聚类分析，识别出最有价值客户。


### 航空公司客户价值信息挖掘主要包話以下步骤：


（1）从航空公司的数据源中进行选择性抽取与新增数据抽取分别形成历史数据和增量数据；

（2）对步骤1）中形成的两个数据集进行数据探索分析和预处理，包括数据缺失值和异常值分析，数据属性的规约、清洗和变换；

（3）利用步骤2）中的处理的数据进行建模，基于旅客价值的LRFMC模型进行客户分类，对各个客户群进行特征分析，识别出最有价值的客户；

（4）针对模型结果得到不同价值的客户，采用不同的营销手段，提供定制化的服务。

# 2. 数据探索分析

In [13]:
# 对数据进行基本的探索，返回缺失值个数以及最大值最小值等
import pandas as pd
 
datafile = "./data/air_data.csv"  #原始数据
resultfile = "./tmp/explore.xls"   #数据探索结果总结表
 
data = pd.read_csv(datafile,encoding = "utf-8")  #读取原始数据，指定UTF-8编码（需要用文本编辑器将数据装换为UTF-8编码）

# 包括对数据的基本描述，percentiles参数是指定计算多少的分位数表（如1/4分位数、中位数等）；T是转置，转置后更方便查
explore = data.describe(percentiles=[],include="all").T
 
#describe()函数自动计算非空值数，需要手动计算空值数
explore["null"] = len(data) - explore["count"]
 
explore = explore[["null","max","min"]]
explore.columns = [u"空数值",u"最大值",u"最小值"] #对结果统计的表头重命名

# 这里只选取部分探索结果。
# describe()函数自动计算的字段有count（非空值数）、unique（唯一值数）、top（频数最高者）、
# freq（最高频数）、mean（平均值）、std（方差）、min（最小值）、50%（中位数）、max（最大值）
# print(explore)
 
explore.to_excel(resultfile) ##导出结果

# 3. 数据预处理